In [187]:
import deal_financials
import statsmodels.api as sm
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [188]:
class FinancialDtCleaner():
    
    """
    Data Validation:
        - Clean Duplicates
        - Adjust Records PublishDate
        - Convert Cumsum value to Single Quarter value, and TTM value
        - Assure Continuity of Records
    """
    
    bls_cols = [ 'totasset','totliab', 'totalnoncliab', 'prest', 'perbond', 'minysharrigh',
       'totalcurrliab', 'paresharrigh', 'righaggr']
    ic_cols = ['bizinco', 'incotaxexpe','netprofit', 'inteexpe', 'bizcost', 'parenetp']
    cf_cols = ['cashnetr','bizcashinfl', 'mananetr', 'acquassetcash', 'fixedassetnetc', 'capex',
       'equfinalbala']
    
    def __init__(self,df):
        self._df = df 
        self._df=  self._format(self._df)
        
    def _format(self,_df):
        _df.columns = _df.columns.str.lower()
        _df['reportdatetype'] = _df['reportdatetype'].astype('str')
        _df['symbol'] = _df['symbol'].apply(lambda x: str(x).zfill(6))
        _df['enddate'] = pd.to_datetime(_df['enddate'],format="%Y%m%d")
        _df['publishdate'] = pd.to_datetime(_df['publishdate'],format="%Y%m%d")
        return _df
        
    def _clean_duplicates(self,_df):
        _df.drop_duplicates(subset=['symbol','enddate'],keep='last',inplace=True)
        return _df
        
    def _convert_to_ttm(self,_df):
        cols = self.ic_cols + self.cf_cols
        _ttm_df = deal_financials.convert_to_ttm(_df,cols)
        return _ttm_df
    
    def _convert_to_single_quarter(self, _df):
        cols = self.ic_cols + self.cf_cols
        _quarter_df = deal_financials.convert_to_single_quarter(_df, cols)
        return _quarter_df
    
    def _adjust_publish_date(self,_df):
        _df = deal_financials.remove_error_rows(_df)
        return _df

    def _ensure_continuity(self,_df):
        _df = deal_financials.ensure_continuity(_df)
        return _df 
    
    def _preprocess(self,_df):
        _df = self._format(_df)
        _df = self._clean_duplicates(_df)
        _df = self._adjust_publish_date(_df)
        _df = self._ensure_continuity(_df)
        _df.sort_values(by=['symbol','enddate'],inplace=True)
        _df.reset_index(drop=True,inplace=True)
        return _df
    
    def get_sq_df(self,_df):
        _quarter_df = self._convert_to_single_quarter(_df)
        return _quarter_df
    
    def get_ttm_df(self,_df):
        _ttm_df = self._convert_to_ttm(_df)
        return _ttm_df

In [189]:
df = pd.read_csv('/Users/apple/Documents/test/financials.csv',encoding='gbk')

In [190]:
fl = FinancialDtCleaner(df)
fl._df = fl._preprocess(_df=fl._df)

In [191]:
fl.mrq_df = fl.get_sq_df(fl._df)
fl.ttm_df = fl.get_ttm_df(fl._df)

In [193]:
def get_roe(group):
    group['prest'] = group['prest'].fillna(0)
    group['perbond'] = group['perbond'].fillna(0)
    group['netasset'] = group['paresharrigh']-group['prest']-group['perbond']
    group['roe'] = group['netprofit']/((group['netasset'].shift(1))+group['netasset'])/2
    return group['roe']

def get_roa(group):
    group['roe'] = group['netprofit']/((group['totasset'].shift(1))+group['totasset'])/2
    return group['roe']


fl.ttm_df['roe_ttm']  = fl.ttm_df.groupby('symbol').apply(get_roe).reset_index()['roe']
fl.mrq_df['roe_mrq']  = fl.mrq_df.groupby('symbol').apply(get_roe).reset_index()['roe']
fl.ttm_df['roa_ttm'] = fl.ttm_df.groupby('symbol').apply(get_roa).reset_index()['roe']
fl.mrq_df['roa_mrq'] = fl.mrq_df.groupby('symbol').apply(get_roa).reset_index()['roe']

In [194]:
fl.mrq_df.to_csv("/Users/apple/Documents/qsquant/source/stock/financials/mrq_financials.csv",encoding = 'gbk')
fl.ttm_df.to_csv("/Users/apple/Documents/qsquant/source/stock/financials/ttm_financials.csv",encoding = 'gbk')

In [197]:
class FinancialFactor():
    
    """
    
    
    Financial df : stores the financial data of all the stocks in the market,
    
    | symbol | publishdate |   enddate  | reportdatetype | feature1 | feature2 | ...
    | ------ | ----------- | ---------- | -------------- | -------- | -------- | ---
    | 000001 | 2019-01-01  | 2018-12-30 | '1'/'2'/'3'    | 1.0      | 2.0      | ...
    
    
    A great deal of financial data is reported quarterly, and the publish date is not fixed.
    
    ** Some companies publish their Q4 report on the same date as their Q1 report of the next year. **
    ** We need to remove the Q4 report if this happens. **
    ** But we need to keep the Q4 report if we use the Q4 report to calculate **
    
    Factor:
    
    | publishdate | 000001 | 000002 | 000003 | ...
    | ----------- | ------ | ------ | ------ | ---
    | 2019-01-01  | 1.0    | 2.0    | 3.0    | ...
    
    """
    
    def __init__(self,_df) -> None:
        self._df = self._format(_df)
        # Function to remove Q4 report if both Q4 and Q1 reports are published on the same date
        self._df_wo_q4 = self._remove_q4_if_q1(self._df)
        
    def _format(self,_df):
        _df.columns = _df.columns.str.lower()
        _df['reportdatetype'] = _df['reportdatetype'].astype('str')
        _df['symbol'] = _df['symbol'].apply(lambda x: str(x).zfill(6))
        return _df
    
    def _remove_q4_if_q1(self,_df):
        return _df.drop_duplicates(subset=['symbol','publishdate'],keep='last') 
    
    def _get_field_df(self,_col):
        _col_df = self._df_wo_q4.pivot(index ='publishdate', columns = 'symbol', values=_col)
        _col_df = _col_df.ffill()
        return _col_df

    def _calc_sql_slope_growth(self,col='bizinco', window_q=20):
        
        def _calc_sql_slope_growth_group(group, col, window_q):
            
            coefs = []
            
            for i in range(0, len(group)):
                if i < window_q:
                    coefs.append(np.nan)
                    continue
                
                y = group[col].iloc[i-window_q:i].dropna().values
                
                if len(y) < window_q/2-2:
                    coefs.append(np.nan)
                    continue
                x = np.arange(1,len(y)+1)
                x = sm.add_constant(x)
                model = sm.OLS(y, x)
                results = model.fit()
                coef = results.params[1]/ y.mean()
                coefs.append(coef)
                    
            return pd.Series(coefs, index=group.index)
        
        _fin_idx_df = self._df[['symbol','publishdate']]
        _fin_idx_df[0] = self._df.groupby('symbol').apply(lambda group: _calc_sql_slope_growth_group(group,col,window_q)).reset_index()[0]
        _fin_idx_df = self._remove_q4_if_q1(_fin_idx_df)
        _f = _fin_idx_df.pivot(index='publishdate',columns='symbol',values=0)
        return _f

In [196]:
mrq_df = pd.read_csv("/Users/apple/Documents/qsquant/source/stock/financials/mrq_financials.csv",encoding = 'gbk',index_col=[0])
ttm_df = pd.read_csv("/Users/apple/Documents/qsquant/source/stock/financials/ttm_financials.csv",encoding = 'gbk',index_col=[0])

In [176]:
FinFactor_ttm= FinancialFactor(ttm_df)
FinFactor_mrq= FinancialFactor(mrq_df)

In [155]:
# EGRO = FinFactor_ttm._calc_sql_slope_growth(col='parenetp', window_q=20)

In [156]:
# SGRO = FinFactor_ttm._calc_sql_slope_growth(col='bizinco', window_q=20)

In [157]:
# EGRO.to_pickle('/Users/apple/Documents/qsquant/processed/stock/EGRO.pkl')
# SGRO.to_pickle('/Users/apple/Documents/qsquant/processed/stock/SGRO.pkl')

In [158]:
balance_cols = FinFactor_ttm._df.columns[FinFactor_ttm._df.columns.get_loc('totasset'):FinFactor_ttm._df.columns.get_loc('enddate_quarter')]

In [148]:
for i in balance_cols:
    field = FinFactor_ttm._get_field_df(i)
    field.to_pickle('/Users/apple/Documents/qsquant/processed/stock/{}.pkl'.format(i))

In [185]:
cf_ic_cols = FinFactor_ttm._df.columns[FinFactor_ttm._df.columns.get_loc('bizinco'):]
print(cf_ic_cols)
for field in cf_ic_cols:
    field_df = FinFactor_ttm._get_field_df(field)
    if 'ttm' not in field:
        field_df.to_pickle('/Users/apple/Documents/qsquant/processed/stock/{}_ttm.pkl'.format(field))
    else:
        field_df.to_pickle('/Users/apple/Documents/qsquant/processed/stock/{}.pkl'.format(field))

Index(['bizinco', 'incotaxexpe', 'netprofit', 'inteexpe', 'bizcost',
       'parenetp', 'cashnetr', 'bizcashinfl', 'mananetr', 'acquassetcash',
       'fixedassetnetc', 'capex', 'equfinalbala', 'roe_ttm', 'roa_ttm'],
      dtype='object')


In [186]:
cf_ic_cols = FinFactor_mrq._df.columns[FinFactor_mrq._df.columns.get_loc('bizinco'):]
print(cf_ic_cols)
for field in cf_ic_cols:
    field_df = FinFactor_mrq._get_field_df(field)
    if 'mrq' not in field:
        field_df.to_pickle('/Users/apple/Documents/qsquant/processed/stock/{}_mrq.pkl'.format(field))
    else :
        field_df.to_pickle('/Users/apple/Documents/qsquant/processed/stock/{}.pkl'.format(field))

Index(['bizinco', 'incotaxexpe', 'netprofit', 'inteexpe', 'bizcost',
       'parenetp', 'cashnetr', 'bizcashinfl', 'mananetr', 'acquassetcash',
       'fixedassetnetc', 'capex', 'equfinalbala', 'roe_mrq', 'roa_mrq'],
      dtype='object')


In [198]:
import sys
sys.path.append('..')
from shanf.datakit.DtSource import DtSource 

23.5.4.60008


In [207]:
fin_derivaties = pd.read_csv('/Users/apple/Documents/qsquant/source/stock/financials/fin_derivatives.csv',encoding='gbk')

In [208]:
fin_derivaties.columns = fin_derivaties.columns.str.lower()

In [209]:
fin_derivaties

,symbol,tradedate,pettm,pb,psttm,pcttm,dy
0,600018,20061025,1764.3292,41.9734,155.0247,NaN,NaN
1,600018,20061025,1764.3292,41.9734,155.0247,NaN,NaN
2,600018,20061024,1764.3292,41.9734,155.0247,NaN,NaN
3,600018,20061024,1764.3292,41.9734,155.0247,NaN,NaN
4,600018,20061023,1764.3292,41.9734,155.0247,NaN,NaN
...,...,...,...,...,...,...,...
12157144,301043,20230804,36.6366,2.5816,4.1695,27.1231,2.0232
12157145,300964,20230804,85.0780,3.4541,6.4082,34.4072,NaN
12157146,688787,20230804,785.9214,6.1657,20.6289,1568.7106,0.5139
12157147,301045,20230804,244.9893,3.1819,4.6421,18.7096,NaN


In [202]:
fin_derivaties.loc[:,:] = fin_derivaties[~fin_derivaties.duplicated()]

In [203]:
fin_derivaties = fin_derivaties[fin_derivaties['symbol'].notna()]

In [204]:
fin_derivaties['tradedate'] = pd.to_datetime(fin_derivaties['tradedate'],format="%Y%m%d")
fin_derivaties['symbol'] = fin_derivaties['symbol'].astype(int).astype(str).apply(lambda x: x.zfill(6))

In [205]:
dy = fin_derivaties.pivot(index='tradedate',columns='symbol',values='dy')

In [206]:
dy

symbol,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,839680,839729,839946,870436,871245,871396,871553,871642,871981,872925
tradedate,,,,,,,,,,,,,,,,,,,,,
2006-01-04,NaN,NaN,NaN,NaN,1.7564,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-05,NaN,NaN,NaN,NaN,1.7564,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-06,NaN,NaN,NaN,NaN,1.7564,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-09,NaN,NaN,NaN,NaN,1.7564,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-10,NaN,NaN,NaN,NaN,1.7564,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.9285,NaN,NaN,3.9086,NaN,NaN,NaN,NaN
2022-04-06,1.0982,5.8906,NaN,NaN,3.4525,NaN,NaN,0.2712,NaN,2.7553,...,NaN,5.6689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-07,1.1056,5.9780,NaN,NaN,3.7258,NaN,NaN,0.2788,NaN,3.0619,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
dy.to_pickle('/Users/apple/Documents/qsquant/processed/stock/dy.pkl')